# Sequence Labeling

In this assignment, you will work on the [MeasEval](https://competitions.codalab.org/competitions/25770) shared task that was part of SemEval-2021. The goal of **MeasEval** is  the extraction of counts, measurements, and related context from scientific documents. The task is a complex problem that involves solving a number of steps that range from identifying quantities and units of measurement to identify relationships between them. For this assignment, you will focus only on the *Quantity* recognition step:

*  Given a paragraph from a scientific text, identify all spans containing quantities like *12 kg*. This problem can be approached as a Sequence Labeling task.

You will develop a Recurrent Neural Network with [keras](https://keras.io/), a high-level Deep Learning API written in **Python** that provides a user-friendly interface for the [TensorFlow](https://www.tensorflow.org/) library, one of the most popular low-level Deep Learning frameworks. You will use the following objects and functions:

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, TimeDistributed
from tensorflow.keras.initializers import Constant
from sklearn.metrics import classification_report

In [2]:
# Enable Eager mode to avoid issues with TimeDistributed layer in Keras 3
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

When working with Neural Networks, there are a large number of random operations such as initializing the weights of the network, shuffling the data for training, or choosing samples. This causes that different training runs of the same model can lead to different results. To ensure reproducibility, i.e. obtaining the same results in the different runs, the random number generator must be initialized with a fixed value known as seed. In **keras**, this can be done as follows:

In [3]:
seed = 42
set_random_seed(seed)

When developing a model, if the results you get are not as expected, try re-initializing the seed by running the cell above before compiling and training the model.

> **Note!** With models as complex as Neural Networks, reproducibility is susceptible to factors such as software versions or the hardware on which the models are run. Even with seed initialization, there may be slight differences in the results.

Working with Neural Networks also involves defining a number of hyperparameters that set the configuration of the model. Finding the appropriate hyperparameter values requires training the model with different combinations and testing them on the development set. This hyperparameter tuning is a costly process that needs multiple rounds of experimentation. However, for this assignments, you will use the following values:

In [4]:
maxlen = 130  # Maximum length of the input sequence accepted by the model
epochs = 5  # Number of epochs to train the model
batch_size = 64  # Number of examples used per gradient update
embedding_dim = 300  # Dimension of the embeddings
rnn_units = 256  # Number of units per RNN layer

Training a Deep Learning model with a large train set can be a time-consuming process, as the model needs to iterate over the entire set multiple times, often requiring significant computational resources. During the implementation of the model, it is often a good practice to use only a subset of the training data. This allows a faster debugging of the code. Set the `shrink_dataset` variable as `True` when a faster training is required and set it as `False` to train the model on the whole train set:

In [5]:
shrink_dataset = False

Although the value of this variable does not affect the tests that will evaluate your code, the output examples distributed throughout this notebook are based on a `shrink_dataset` variable set as `False`.

The train set for the assignment consists of 248 articles with 1366 sentences in total. The test set contains 136 articles with 848 sentences. A development set with 65 documents and 459 sentences is also provided. The dataset is annotated at the token level following a BIO schema with 3 labels: *B-Quantity*, *I-Quantity* and *O*.  The dataset can be loaded into three `DataFrames` as follows:

In [6]:
def load_data(data_path, shrink_dataset, seed):
    data = pd.read_csv(data_path, sep="\t", encoding="utf8").dropna()
    if shrink_dataset:
        sample = data[["docId",  "sentId"]].drop_duplicates().sample(frac=0.2, random_state=seed)
        data = pd.merge(data, sample, on=["docId", "sentId"])
    return data

In [11]:
train_data = load_data("data/train.tsv", shrink_dataset, seed)
dev_data = load_data("data/trial.tsv", shrink_dataset, seed)
test_data = load_data("data/eval.tsv", shrink_dataset, seed)
train_data[(train_data.docId == "S0378383912000130-3601") & (train_data.sentId == 3)]

,docId,sentId,word,lemma,label
22276,S0378383912000130-3601,3,The,the,O
22277,S0378383912000130-3601,3,experiments,experiment,O
22278,S0378383912000130-3601,3,involved,involve,O
22279,S0378383912000130-3601,3,two,two,B-Quantity
22280,S0378383912000130-3601,3,beach,beach,I-Quantity
22281,S0378383912000130-3601,3,materials,material,I-Quantity
22282,S0378383912000130-3601,3,with,with,O
22283,S0378383912000130-3601,3,nominal,nominal,O
22284,S0378383912000130-3601,3,sediment,sediment,O
22285,S0378383912000130-3601,3,diameters,diameter,O


The `DataFrames` created include the lemmatization of words in the `lemma` columns. You will use the lemmas as the input of the model.

## Data Pre-processing

In this assignment, you will have to implement some steps to pre-process and obtain a representation of the data. You will implement a model with an `Embedding` lookup table as the input layer, so the tokens of the input sentences should be represented as indexes. The target labels should also be represented in similar way. Besides, as one would expect, the sentences in the **MeasEval** dataset have different lengths. However, the input for a Deep Learning model is a batch of examples (in this case, sentences) in the form of a single tensor which requires that all examples in the batch must have the same length. Therefore, the sentences should be padded or truncated to a specific length.

> **Note!** For this particular task, the `maxlen` value provided to you guarantees that padding is sufficient to make all sentences the same length without the need for truncation.

This first of these pre-processing steps will be to obtain both a vocabulary and the set of labels from the train set. The vocabulary should be the list of unique lemmas and must include the special tokens `[PAD]`, that will be used for padding the sequences, and `[UNK]`, that will be used to represent out-of-vocabulary words. Along with the vocabulary and the label set, you will also have to build a dictionary mapping each lemma to its position in the vocabulary and a dictionary mapping each label to its position in the label set. These dictionaries will be used later to obtain the representation of the input and output of the model. The text is already tokenized and lemmatized which will help in this task.

You must complete the code for the `get_vocabulary` function that takes as input the `DataFrame` containing the train set. The function should create a list with the all the unique lemmas and include the special tokens `[PAD]` and `[UNK]` in the first two positions. Similarly, the function should create a list with the unique labels with the special token `[PAD]` in the first position. The **pandas** library provides some [functions](https://pandas.pydata.org/docs/reference/index.html) that may help you. Along with those lists, `get_vocabulary` should return the dictionaries mapping the lemmas and the labels to their corresponding positions. In total, the vocabulary and the label set should have 5508 and 4 items respectively:

> Vocabulary size: 5508  
Vocabulary first 5 lemmas: ['[PAD]', '[UNK]', 'datum', 'be', 'draw']  
Vocabulary dictionary: {'[PAD]': 0, '[UNK]': 1, 'datum': 2, 'be': 3, 'draw': 4}  
>
>Labels size: 4  
Labels: ['[PAD]', 'O', 'B-Quantity', 'I-Quantity']  
Labels dictionary: {'[PAD]': 0, 'O': 1, 'B-Quantity': 2, 'I-Quantity': 3}  


In [ ]:
def get_vocabulary(train_data):
    """Creates vocabulary and label dictionaries."""
    vocab_list = ['[PAD]', '[UNK]']
    unique_vocab = train_data['lemma'].unique()
    vocab_list += list(unique_vocab)
    word2idx = {vocab : idx for idx, vocab in enumerate(vocab_list)}
    label_list = ['[PAD]']
    unique_labels = train_data['label'].unique()
    label_list += list(unique_labels)
    label2idx = {label : idx for idx, label in enumerate(label_list)}
    return vocab_list, word2idx, label_list, label2idx

In [15]:
vocab, word2idx, labels, label2idx = get_vocabulary(train_data)
vocab_size = len(vocab)
label_size = len(labels)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary first 5 words: {vocab[:5]}")
print(f"Vocabulary dictionary: { {w: word2idx[w] for w in vocab[:5]}}")
print("")
print(f"Labels size: {label_size}")
print(f"Labels: {labels}")
print(f"Labels dictionary: {label2idx}")

Vocabulary size: 5508
Vocabulary first 5 words: ['[PAD]', '[UNK]', 'datum', 'be', 'draw']
Vocabulary dictionary: {'[PAD]': 0, '[UNK]': 1, 'datum': 2, 'be': 3, 'draw': 4}

Labels size: 4
Labels: ['[PAD]', 'O', 'B-Quantity', 'I-Quantity']
Labels dictionary: {'[PAD]': 0, 'O': 1, 'B-Quantity': 2, 'I-Quantity': 3}


Since the *Quantity* recognition task is a Sequence Labeling problem, the input for the model must be the sequence of lemmas in the sentence and the output the sequence of labels. Therefore, the train, development and test `DataFrames` must be reformated by aggregating the data corresponding to each sentence. The `integrate_sentences` will do this for you. The output of `integrate_sentences` is a `DataFrame` with a row for each sentence and the columns `lemmas` and `labels` that contain the list of lemmas and the list of labels of the sentences respectively.  

In [16]:
def integrate_sentences(data):
    agg_func = lambda s: [s['lemma'].values.tolist(), s['label'].values.tolist()]
    data = data.groupby(["docId", "sentId"], sort=False).apply(agg_func, include_groups=False).reset_index().rename(columns={0: 'lemmas_labels'})
    data['lemmas'] = data.apply(lambda x: x['lemmas_labels'][0], axis=1)
    data['labels'] = data.apply(lambda x: x['lemmas_labels'][1], axis=1)
    data = data.drop(columns="lemmas_labels")
    return data

In [17]:
train_examples = integrate_sentences(train_data)
dev_examples = integrate_sentences(dev_data)
test_examples = integrate_sentences(test_data)
pd.set_option('display.max_colwidth', None)
train_examples[(train_examples.docId == "S0378383912000130-3601") & (train_examples.sentId == 3)]

,docId,sentId,lemmas,labels
767,S0378383912000130-3601,3,"[the, experiment, involve, two, beach, material, with, nominal, sediment, diameter, of, 1.5, mm, and, 8.5, mm, .]","[O, O, O, B-Quantity, I-Quantity, I-Quantity, O, O, O, O, O, B-Quantity, I-Quantity, I-Quantity, I-Quantity, I-Quantity, O]"


The dataset is now ready for you to get the numerical representation of both input and output. You must perform two steps to process the sequence of lemmas and the sequence of labels:
1. For each sentence, translate each lemma or label to its corresponding index using the `word2idx` and `label2idx` dictionaries. In case the lemma is not found in `word2idx`, use the index of the `[UNK]` token instead.
2. Pad both the sequences of lemmas and the sequences of labels to the same length as defined by the `maxlen` variable. For this, you should use the [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences) function with its default padding strategy. This function uses `0` as the default padding value which corresponds to the index of the `[PAD]` token in the vocabulary.   

You must complete the code for the `format_examples` function. This function takes as input a `DataFrame` in the format returned by `integrate_sentences`, the `word2idx` and `label2idx` dictionaries, and the `maxlen` variable. The function must run the steps described above and return a **numpy** array with the processed lemma sequences and a **numpy** array with the processed label sequences that will be used as input and output of the model respectively. Applying `format_examples` to the train, development and test sets should result on 6 arrays with the following shapes:

>Shape of train input data :  (1366, 130)  
Shape of train output data :  (1366, 130)  
Shape of development input data :  (459, 130)  
Shape of development output data :  (459, 130)  
Shape of test input data :  (848, 130)  
Shape of test output data :  (848, 130)  

In [ ]:
def format_examples(data, word2idx, label2idx, maxlen):
    lemmas_seq = data['lemmas']
    labels_seq = data['labels']
    
    x = [[word2idx.get(word, word2idx['[UNK]']) for word in seq] for seq in lemmas_seq]
    y = [[label2idx.get(label, label2idx['[PAD]']) for label in seq] for seq in labels_seq]
    
    x = pad_sequences(x, maxlen=maxlen, padding='post')
    y = pad_sequences(y, maxlen=maxlen, padding='post')
    
    return x, y

In [19]:
x_train, y_train = format_examples(train_examples, word2idx, label2idx, maxlen)
x_dev, y_dev = format_examples(dev_examples, word2idx, label2idx, maxlen)
x_test, y_test = format_examples(test_examples, word2idx, label2idx, maxlen)
print("Shape of train input data: ", x_train.shape)
print("Shape of train output data: ", y_train.shape)
print("Shape of development input data: ", x_dev.shape)
print("Shape of development output data: ", y_dev.shape)
print("Shape of train input data: ", x_test.shape)
print("Shape of test output data: ", y_test.shape)

Shape of train input data:  (1366, 130)
Shape of train output data:  (1366, 130)
Shape of development input data:  (459, 130)
Shape of development output data:  (459, 130)
Shape of train input data:  (848, 130)
Shape of test output data:  (848, 130)


## Recurrent Neural Network

There are three ways to create a neural network with **keras**: using the [Functional API](https://keras.io/guides/functional_api/), by [Model subclassing](https://keras.io/guides/making_new_layers_and_models_via_subclassing/) or creating a [Sequential model](https://keras.io/guides/sequential_model/). In this assignment, you will use the latter option. A `Sequential` model is a straightforward approach to build simple neural networks by stacking the layers. You will construct a RNN with the following 3 layers:
1. An [Input](https://keras.io/api/layers/core_layers/input/) layer where the length of the input sequences is set to `maxlen`.
2. An [Embedding](https://keras.io/api/layers/core_layers/embedding/) layer with an input dimension equal to the vocabulary size and an embedding dimension defined by `embedding_dim`. The layer must also mask out the padding values so that they are not considered when computing the loss.
3. A Bidirectional [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/) layer with a number of units determined by `rnn_units`. Since you are working on Sequence Labeling, the `LSTM` must return outputs for the full sequence. To make it Bidirectional, the `LSTM` must be wrapped by a [Bidirectional](https://keras.io/api/layers/recurrent_layers/bidirectional/) layer.
4. A [Dense](https://keras.io/api/layers/core_layers/dense/) layer with a number of units equal to the number of labels and a `softmax` activation function.  Since you are working on Sequence Labeling, the `Dense` layer must be wrapped by a [TimeDistributed](https://keras.io/api/layers/recurrent_layers/time_distributed/) layer.

You must complete the code for the `create_model` function. This function takes as input the size of the vocabulary, the number of labels and the `maxlen`, `embedding_dim` and `rnn_units` hyperparameters. The function must create a RNN according to the configuration described above. Read carefully all the linked documentation to learn how to create such a model.  Any option not mentioned in the description should be kept with its default value. The summary of the resulting model should look like:


> <pre>
> Model: "sequential_1"
> __________________________________________________________________________________________
> Layer (type)                           Output Shape                        Param #       
> ==========================================================================================
> embedding_1 (Embedding)               (None, 130, 300)                    1652400       
>                                                                                          
> bidirectional_1 (Bidirectional)       (None, 130, 512)                    1140736       
>                                                                                          
> time_distributed_1 (TimeDistributed)  (None, 130, 4)                      2052          
>                                                                                          
> ==========================================================================================
> Total params: 2,795,188
> Trainable params: 2,795,188
> Non-trainable params: 0
> __________________________________________________________________________________________
> </pre>

Before returning the model, the `create_model` function should [compile](https://keras.io/api/models/model_training_apis/#compile-method) it using `'sparse_categorical_crossentropy'` as the loss function, `'adam'` as the optimizer and `'sparse_categorical_accuracy'` as a metric to evaluate the model during training.

In [ ]:
def create_model(vocab_size, label_size, maxlen, embedding_dim, rnn_units):
    #
    #  REPLACE THE pass STATEMENT WITH YOUR CODE
    #
    pass

In [ ]:
model = create_model(vocab_size, label_size, maxlen, embedding_dim, rnn_units)
model.summary(line_length=90)

Once the data has been processed and the model has been compiled, you can proceed to train it.

You must complete the `train_model` function. The function takes as input the model created by `create_model`, the train input and output obtained by `format_examples` as well as the development input and output produced by the same function. The function also takes the `batch_size` and `epochs` hyperparameters. The function should train the model on the training data using those hyperparameters. During the training, `train_model` should evaluate the loss and any model metrics on the development data. With `shrink_dataset = False`, the training will take several minutes.

In [ ]:
def train_model(model, x_train, y_train, x_dev, y_dev, batch_size, epochs):
    #
    #  REPLACE THE pass STATEMENT WITH YOUR CODE
    #
    pass

In [ ]:
train_model(model, x_train, y_train, x_dev, y_dev, batch_size, epochs)

After training, the model can be used to make predictions on unlabeled data using the [predict](https://keras.io/api/models/model_training_apis/#predict-method) method.

You must complete the code for the `make_predictions` function. The functions takes as input the model already trained, the test input data produced by `format_examples` and the `batch_size` hyperparameter. The function must run the `predict` method on the input data using batches of size equal to `batch_size`. The `predict` method will return a **numpy** array with 3 axes: `(number of sentences, maxlen, label_size)`. For each token in each sentence, `predict` returns a vector with the probabilities predicted for every label. The output of `make_predictions` must include only the index of the label with the highest probability for each token. For example, if the prediction for one token is the vector `[0.04974193, 0.1511916, 0.65180656, 0.14725993]`, the output for that token should be `2`. For this, you can apply the [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) method along the last axis of the **numpy** array.

In [ ]:
def make_predictions(model, x_test, batch_size):
    #
    #  REPLACE THE pass STATEMENT WITH YOUR CODE
    #
    pass

In [ ]:
predictions = make_predictions(model, x_test, batch_size)

Since the predictions are now label indexes, they can be translated to the corresponding label by accessing the `labels` list. The `predictions_to_labels` functions iterates over all the sequences in the test set and translates the prediction of each token to the corresponding label. The function skips the padding tokens. The new format of the predictions can be stored in the `prediction` column of the test `DataFrame`.

In [ ]:
def predictions_to_labels(predictions, x_test, labels):
    pred_labels = []
    for pred_seq, x_seq in zip(predictions, x_test):
        pred_seq_labels = [labels[p] for p, x in zip(pred_seq, x_seq) if x!=0]
        pred_labels.extend(pred_seq_labels)
    return pred_labels

In [ ]:
test_data['prediction'] = predictions_to_labels(predictions, x_test, labels)
test_data[(test_data.docId == "S0038071711004354-1624") & (test_data.sentId == 2)]

Although it is not the usual way of evaluating Sequence Labeling tasks, **MeasEval** uses a metric based on the Reading Comprehension *Macro-Averaged F1*. This metric measures the amount of overlapping tokens between the predictions and the true labels. In this assignment, we will approximate this metric by evaluating how many tokens belonging to a *Quantity* are captured by the model. This is done by the `evaluate` function. For the model trained above, the result of this evaluation should look like:

> <pre>
>               precision    recall  f1-score   support
>
>     Quantity       0.86      0.73      0.79      1263
>
>    micro avg       0.86      0.73      0.79      1263
>    macro avg       0.86      0.73      0.79      1263
> weighted avg       0.86      0.73      0.79      1263
> </pre>

In [ ]:
def evaluate(data):
    labels = data.apply(lambda x: x['label'].replace("B-", "").replace("I-", ""), axis=1).values
    predictions = data.apply(lambda x: x['prediction'].replace("B-", "").replace("I-", ""), axis=1).values
    print(classification_report(labels, predictions, labels=["Quantity"]))

In [ ]:
evaluate(test_data)

## Pre-trained Word Embeddings

Initializing neural networks with pre-trained word embeddings has a significant impact on many NLP tasks. In the following exercise, you will experiment whether this is also the case for *Quantity* recognition using **GloVe**. You can refer to the following tutorial to learn how to complete this exercise with **keras**: [Using pre-trained word embeddings
](https://keras.io/examples/nlp/pretrained_word_embeddings/)


First, the `load_embeddings` function will load **GloVe** and return a dictionary mapping words to their embeddings.

In [ ]:
def load_embeddings(glove_path):
    embedding_index = {}
    with open(glove_path, encoding="utf8") as glove_file:
        for line in glove_file:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embedding_index[word] = coefs
    return embedding_index

In [ ]:
glove_path = f"glove/glove.6B.{embedding_dim}d.txt"
embedding_index = load_embeddings(glove_path)

To initialize the `Embedding` layer with the **GloVe** embeddings, you have to create a matrix with `(vocab_size, embedding_dim)` dimensions. The *i-th* row in the matrix corresponds to the *i-th* lemma in the vocabulary and contains the **GloVe** embedding for that lemma.

You must complete the code for the `create_embedding_matrix` function. The function takes the embedding dictionary created by `load_embeddings`, the vocabulary dictionary, the size of the vocabulary and the `embedding_dim` hyperparameter. The function should initialize a `(vocab_size, embedding_dim)` **numpy** array with zeros and then replace each row with the appropriate **GloVe** embedding if the corresponding lemma exists in the embedding dictionary. For example, the embedding for "*statistic*" should exist in the resulting `embedding_matrix`:
> <pre>
> array([ 0.1085   ,  0.82802  ,  0.10672  ,  0.0094136, -0.30441  ,
>         0.75618  , -0.14705  , -0.15469  , -0.97372  , -0.60413  , ...,
>         0.0066196,  0.17067  ,  0.16759  ,  0.63236  ,  0.22037  ,
>        -0.2467   , -0.20493  , -0.55613  ,  0.013659 , -0.35642  ])
</pre>

In [ ]:
def create_embedding_matrix(embedding_index, word2idx, vocab_size, embedding_dim):
    #
    #  REPLACE THE pass STATEMENT WITH YOUR CODE
    #
    pass

In [ ]:
embedding_matrix = create_embedding_matrix(embedding_index, word2idx, vocab_size, embedding_dim)
with np.printoptions(linewidth=75, precision=7, edgeitems=10, threshold=10):
    display(embedding_matrix[word2idx["statistic"]])

Finally, you can create a new model and load the pre-trained word embedding matrix into the `Embedding` layer.

You must complete the code for the `create_model_with_embeddings` function. This function takes as input the size of the vocabulary, the number of labels, the `maxlen`, `embedding_dim` and `rnn_units` hyperparameters, and the embedding matrix created by `create_embedding_matrix`. The function should construct, compile and return a RNN equal to the one built in `create_model` with the only difference that the `Embedding` layer must be initialized with the embedding matrix. Use the [Constant](https://keras.io/api/layers/initializers/) initializer for this purpose. For this task, the Embedding layer must be kept trainable so the embeddings can be updated during training.

The initialized weights of the `Embedding` layer for this version of the RNN should look like:
> <pre>
> array([[ 0.      ,  0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ,  0.      ],
>        [ 0.      ,  0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ,  0.      ],
>        [ 0.72004 ,  0.80955 ,  0.7717  , -0.10769 , ..., -0.15423 ,  0.39351 , -0.47083 , -0.60759 ],
>        [-0.33848 ,  0.42841 , -0.10284 , -0.22054 , ...,  0.04933 , -0.79888 , -0.41967 , -0.14039 ],
>        ...,
>        [ 0.1593  ,  0.30671 ,  0.093787,  0.20868 , ...,  1.0097  , -0.76431 , -0.12928 ,  0.29616 ],
>        [ 0.      ,  0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ,  0.      ],
>        [-0.40124 , -0.27991 , -0.42446 , -0.96387 , ...,  0.4937  ,  0.45576 ,  0.61864 , -0.30489 ],
>        [ 0.      ,  0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ,  0.      ]],
>       dtype=float32)
> </pre>

In [ ]:
def create_model_with_embeddings(vocab_size, label_size, maxlen, embedding_dim, rnn_units, embedding_matrix):
    #
    #  REPLACE THE pass STATEMENT WITH YOUR CODE
    #
    pass

In [ ]:
model_with_embeddings = create_model_with_embeddings(vocab_size, label_size, maxlen, embedding_dim, rnn_units, embedding_matrix)
with np.printoptions(linewidth=110, precision=7, edgeitems=4):
    display(model_with_embeddings.get_layer(index=0).get_weights()[0])

Initializing the `Embedding` layer with **GloVe** embeddings should have a positive impact on the model performance for *Quantity* recognition.

> <pre>
>               precision    recall  f1-score   support
>
>     Quantity       0.85      0.79      0.82      1263
>
>    micro avg       0.85      0.79      0.82      1263
>    macro avg       0.85      0.79      0.82      1263
> weighted avg       0.85      0.79      0.82      1263
> </pre>

In [ ]:
train_model(model_with_embeddings, x_train, y_train, x_dev, y_dev, batch_size, epochs)

In [ ]:
predictions_with_embeddings = make_predictions(model_with_embeddings, x_test, batch_size)
test_data['prediction'] = predictions_to_labels(predictions_with_embeddings, x_test, labels)

In [ ]:
evaluate(test_data)